## Model Serving API Call (Task 5)

In [ ]:
pip install flask pandas scikit-learn joblib

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

app = Flask(__name__)

# Load the trained model and scaler
model = joblib.load('credit_scoring_model.joblib')
scaler = joblib.load('scaler.joblib')

@app.route('/predict', methods=['POST'])
def predict():
    # Get the input data from the request
    data = request.json
    
    # Convert the input data to a DataFrame
    input_df = pd.DataFrame([data])
    
    # Ensure the input data has the correct features
    required_features = ['Recency', 'Frequency', 'Monetary', 'Standardization']
    for feature in required_features:
        if feature not in input_df.columns:
            return jsonify({'error': f'Missing required feature: {feature}'}), 400
    
    # Preprocess the input data
    input_scaled = scaler.transform(input_df)
    
    # Make predictions
    prediction = model.predict(input_scaled)
    prediction_proba = model.predict_proba(input_scaled)
    
    # Prepare the response
    response = {
        'prediction': prediction[0],
        'probability': {
            'bad': prediction_proba[0][0],
            'good': prediction_proba[0][1]
        }
    }
    
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True)